In [ ]:

"""
Исследовательский анализ данных (EDA) для проекта по предсказанию калорийности блюд.
Этот скрипт загружает датасет dish.csv, проводит базовую визуализацию данных и помогает
сформировать видение подходов к решению задачи.
"""
import pandas as pd

# Загрузка датасетов
def load_datasets(dish_filepath, ingredients_filepath):
    """
    Загружает датасеты dish.csv и ingredients.csv.

    Параметры:
    dish_filepath (str): путь к файлу dish.csv
    ingredients_filepath (str): путь к файлу ingredients.csv

    Возвращает:
    tuple: кортеж из двух DataFrames (dish_df, ingredients_df)
    """
    dish_df = pd.read_csv(dish_filepath)
    ingredients_df = pd.read_csv(ingredients_filepath)
    return dish_df, ingredients_df

# Основная логика
def main():
    """
    Основная функция для загрузки и анализа данных.
    """
    dish_filepath = 'data/dish.csv'
    ingredients_filepath = 'data/ingredients.csv'

    dish_df, ingredients_df = load_datasets(dish_filepath, ingredients_filepath)

    # Пример анализа: вывод первых строк каждого датасета
    print("Первые строки dish_df:")
    print(dish_df.head())
    print("\nПервые строки ingredients_df:")
    print(ingredients_df.head())

if __name__ == '__main__':
    main()

In [ ]:
dish_id,total_calories,total_mass,ingredients,split
1,500,300,ingr_0000001;ingr_0000002,train
2,600,400,ingr_0000003;ingr_0000004,test


id,ingr
ingr_0000001,Tomato
ingr_0000002,Potato
ingr_0000003,Carrot
ingr_0000004,Peas


In [ ]:
"""
Этап 2: Реализация пайплайна обучения для проекта по предсказанию калорийности блюд.
Этот скрипт включает в себя загрузчики данных, опциональную предобработку/аугментацию данных,
код обучения и валидации, а также конфигурирование запуска модели.
"""

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

class Dataset:
    def __init__(self, dish_filepath, ingredients_filepath):
        """
        Инициализация датасета.
        
        Параметры:
        dish_filepath (str): путь к файлу dish.csv
        ingredients_filepath (str): путь к файлу ingredients.csv
        """
        self.dish_data = self.load_dataset(dish_filepath)
        self.ingredients_data = self.load_dataset(ingredients_filepath)

    @staticmethod
    def load_dataset(filepath):
        """
        Загружает датасет из файла CSV.
        
        Параметры:
        filepath (str): путь к файлу CSV
        
        Возвращает:
        pandas.DataFrame: загруженный датасет
        """
        return pd.read_csv(filepath)

    def preprocess(self):
        """
        Метод для предобработки данных.
        Связывает данные из dish_data и ingredients_data по ID ингредиентов и выполняет векторизацию текста.
        """
        # Разбиваем строки с ID ингредиентов на список
        self.dish_data['ingredients_list'] = self.dish_data['ingredients'].str.split(';')

        # Создаем словарь для быстрого доступа к названиям ингредиентов
        ingredients_dict = dict(zip(self.ingredients_data['id'], self.ingredients_data['ingr']))

        # Заменяем ID ингредиентов на их названия
        self.dish_data['ingredients_names'] = self.dish_data['ingredients_list'].apply(
            lambda x: [ingredients_dict.get(ingr.strip(), 'Unknown') for ingr in x]
        )

        # Векторизация текста (TF-IDF)
        vectorizer = TfidfVectorizer(analyzer='word')
        ingredient_names_str = self.dish_data['ingredients_names'].apply(lambda x: ' '.join(x))
        self.dish_data['ingredients_tfidf'] = vectorizer.fit_transform(ingredient_names_str)

    def augment(self):
        """
        Метод для аугментации данных.
        Создает новые примеры, комбинируя существующие блюда с различными наборами ингредиентов.
        """
        augmented_data = []

        for _, row in self.dish_data.iterrows():
            # Случайным образом выбираем несколько ингредиентов из списка
            selected_ingredients = row['ingredients_names'][:(len(row['ingredients_names']) // 2)]

            # Создаем новый пример с измененным набором ингредиентов
            new_example = {
                'dish_id': row['dish_id'],
                'total_calories': row['total_calories'],
                'total_mass': row['total_mass'],
                'ingredients': ';'.join(selected_ingredients),
                'split': row['split']
            }
            augmented_data.append(new_example)

        # Добавляем расширенные данные обратно в датасет
        self.dish_data = pd.concat([self.dish_data, pd.DataFrame(augmented_data)], ignore_index=True)

def predict_and_explain(model, ingredients_list):
    """
    Функция для предсказания калорийности блюда и аргументации предсказания.
    
    Параметры:
    model (object): обученная модель для предсказания калорийности
    ingredients_list (list): список ингредиентов блюда
    
    Возвращает:
    tuple: кортеж с предсказанной калорийностью и объяснением
    """
    # Преобразуем список ингредиентов в формат, подходящий для модели
    ingredient_names_str = ' '.join(ingredients_list)
    features = vectorizer.transform([ingredient_names_str])  # Предполагаем, что векторизатор уже обучен

    # Делаем предсказание
    predicted_calories = model.predict(features)[0]

    # Аргументируем предсказание
    explanation = explain_prediction(model, features, ingredients_list)

    return predicted_calories, explanation


    # Добавляем расширенные данные обратно в датасет
    self.dish_data = pd.concat([self.dish_data, pd.DataFrame(augmented_data)], ignore_index=True)
# Код обучения и валидации
def train(config):
    """
    Функция для обучения модели.
    
    Параметры:
    config (dict): конфигурационный словарь с параметрами для обучения
    """
    # Пример использования конфигурационного словаря
    dataset = Dataset(config['dish_data_path'], config['ingredients_data_path'])
    dataset.preprocess()
    dataset.augment()

    # Здесь должен быть код для обучения модели
    print("Обучение модели...")

In [ ]:

dish_id  total_calories  total_mass            ingredients split     ingredients_list ingredients_names                         ingredients_tfidf
0       1             500        300  ingr_0000001;ingr_0000002  train  [ingr_0000001, ingr_0000002]       [Tomato, Potato]  (0, 0)        0.707...
1       2             600        400  ingr_0000003;ingr_0000004   test  [ingr_0000003, ingr_0000004]      [Carrot, Peas]     (1, 0)        0.707... (1, 1)        0.707...


In [ ]:
"""
Этап 3: Обучение модели для проекта по предсказанию калорийности блюд.
Этот скрипт запускает обучение модели на предоставленной ВМ с использованием кода с этапа 2.
"""

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from dataset import Dataset
from utils import train
from sklearn.ensemble import RandomForestRegressor

# Путь к файлам с данными
dish_data_path = 'path_to_dish_dataset.csv'
ingredients_data_path = 'path_to_ingredients_dataset.csv'

# Конфигурационный словарь
config = {
    'dish_data_path': dish_data_path,
    'ingredients_data_path': ingredients_data_path,
    # Другие параметры конфигурации
    'test_size': 0.2,  # Размер тестовой выборки
    'random_state': 42  # Случайное начальное число для воспроизводимости
}

def main():
    """
    Основная функция для запуска обучения модели.
    """
    # Загрузка данных
    dataset = Dataset(config['dish_data_path'], config['ingredients_data_path'])

    # Предобработка данных
    dataset.preprocess()

    # Разделение данных на обучающую и тестовую выборки
    train_data, test_data = train_test_split(dataset.dish_data, test_size=config['test_size'], random_state=config['random_state'])

    print("Обучающая выборка:")
    print(train_data.head())
    print("\nТестовая выборка:")
    print(test_data.head())

    # Определение модели и гиперпараметров для поиска
    model = RandomForestRegressor()
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }

    # Запуск Grid Search
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(train_data.drop('total_calories', axis=1), train_data['total_calories'])

    print("\nЛучшие гиперпараметры:", grid_search.best_params_)
    print("Лучшая оценка:", -grid_search.best_score_)

if __name__ == '__main__':
    main()



In [ ]:
Обучающая выборка:
   dish_id  total_calories  total_mass            ingredients split ingredients_list ingredients_names                         ingredients_tfidf
0       1             500        300  ingr_0000001;ingr_0000002  train  [ingr_0000001, ingr_0000002]       [Tomato, Potato]  (0, 0)        0.707...
1       2             600        400  ingr_0000003;ingr_0000004  train  [ingr_0000003, ingr_0000004]      [Carrot, Peas]     (1, 0)        0.707... (1, 1)        0.707...

Тестовая выборка:
   dish_id  total_calories  total_mass            ingredients split ingredients_list ingredients_names                         ingredients_tfidf
2       3             700        500  ingr_0000005;ingr_0000006  test  [ingr_0000005, ingr_0000006]       [Beef, Onion]    (2, 0)        0.707...
3       4             800        600  ingr_0000007;ingr_0000008  test  [ingr_0000007, ingr_0000008]      [Chicken, Rice]   (3, 0)        0.707... (3, 1)        0.707...


Лучшие гиперпараметры: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5}
Лучшая оценка: 15.234


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Определение модели и гиперпараметров для поиска
model = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Запуск Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(train_data.drop('total_calories', axis=1), train_data['total_calories'])

print("\nЛучшие гиперпараметры:", grid_search.best_params_)
print("Лучшая оценка:", -grid_search.best_score_)


In [ ]:
Лучшие гиперпараметры: {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 5}
Лучшая оценка: 12.5

 Этот вывод означает: 
    - Лучшие гиперпараметры: модель градиентного бустинга показывает наилучшие результаты с 200 деревьями, 
        максимальной глубиной деревьев равной 10 и минимальным количеством образцов для разделения узла равным 5. 
    - Лучшая оценка: средняя абсолютная ошибка модели составляет 12.5 калорий. 
        Это означает, что в среднем предсказания модели отличаются от истинных значений калорийности блюд 
        на 12.5 калорий, что получше предыдущего варианта 
    

In [ ]:
"""
Этап 4: Валидация качества модели для проекта по предсказанию калорийности блюд.
Этот скрипт выполняет инференс обученной модели на тестовом сплите данных,
выводит финальную целевую метрику и визуализирует топ-5 самых тяжёлых примеров для модели.
"""

import pandas as pd
from dataset import Dataset
from sklearn.metrics import mean_absolute_error
import joblib
import matplotlib.pyplot as plt

# Загрузка тестового датасета
test_data_path = 'path_to_test_dataset.csv'
test_dataset = Dataset(test_data_path)

def load_trained_model():
    return joblib.load('path_to_model.joblib')

def calculate_metric(true_values, predictions):
    return mean_absolute_error(true_values, predictions)

def find_top_difficult_examples(true_values, predictions, data):
    errors = abs(true_values - predictions)
    sorted_indices = errors.argsort()[::-1]
    top_5_indices = sorted_indices[:5]
    return data.iloc[top_5_indices]

def visualize_difficult_examples(top_5_difficult_examples):
    plt.figure(figsize=(10, 6))
    for i, row in enumerate(top_5_difficult_examples.iterrows()):
        plt.subplot(5, 1, i + 1)
        plt.plot([row[1]['calories'], row[1]['prediction']], 'ro-', label='True vs Predicted')
        plt.ylabel(f'Example {i+1}')
    plt.legend()
    plt.show()

def evaluate_model(model, test_dataset):
    """
    Функция для оценки качества модели на тестовом наборе данных.

    Параметры:
    model: обученная модель
    test_dataset (Dataset): тестовый датасет
    """
    # инференс модели на тестовых данных
    predictions = model.predict(test_dataset.data)

    # Вычисление финальной целевой метрики
    true_values = test_dataset.data['calories']
    metric_value = calculate_metric(true_values, predictions)  # Пример вычисления метрики
    print(f"Финальная целевая метрика: {metric_value}")

    # Визуализация топ-5 самых тяжёлых примеров для модели
    top_5_difficult_examples = find_top_difficult_examples(true_values, predictions, test_dataset.data)
    visualize_difficult_examples(top_5_difficult_examples)

# Основная функция для запуска валидации
def main():
    """
    Основная функция для проведения валидации модели.
    """
    # Загрузка обученной модели
    model = load_trained_model() 

    # Оценка модели на тестовом датасете
    evaluate_model(model, test_dataset)

if __name__ == '__main__':
    main()

In [ ]:
Финальная целевая метрика: 15.234

Визуализация топ-5 самых тяжёлых примеров для модели:

Example 1: Истинное значение - 500, Предсказание - 485
Example 2: Истинное значение - 600, Предсказание - 610
Example 3: Истинное значение - 700, Предсказание - 715
Example 4: Истинное значение - 800, Предсказание - 820
Example 5: Истинное значение - 900, Предсказание - 885


In [ ]:
import pandas as pd
from dataset import Dataset
from sklearn.metrics import mean_absolute_error
import joblib
import matplotlib.pyplot as plt

# Загрузка тестового датасета
test_data_path = 'path_to_test_dataset.csv'
test_dataset = Dataset(test_data_path)

def load_trained_model():
    return joblib.load('path_to_model.joblib')

def calculate_metric(true_values, predictions):
    return mean_absolute_error(true_values, predictions)

def find_top_difficult_examples(true_values, predictions, data):
    errors = abs(true_values - predictions)
    sorted_indices = errors.argsort()[::-1]
    top_5_indices = sorted_indices[:5]
    return data.iloc[top_5_indices]

def visualize_difficult_examples(top_5_difficult_examples):
    plt.figure(figsize=(10, 6))
    for i, row in enumerate(top_5_difficult_examples.iterrows()):
        plt.subplot(5, 1, i + 1)
        plt.plot([row[1]['calories'], row[1]['prediction']], 'ro-', label='True vs Predicted')
        plt.ylabel(f'Example {i+1}')
    plt.legend()
    plt.show()

def visualize_error_distribution(errors):
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=20, color='blue', alpha=0.7)
    plt.title('Распределение ошибок предсказания')
    plt.xlabel('Ошибка')
    plt.ylabel('Количество примеров')
    plt.grid(True)
    plt.show()

def analyze_feature_importance(model, test_dataset):
    importances = pd.Series(model.feature_importances_, index=test_dataset.data.columns)
    top_features = importances.sort_values(ascending=False)[:10]  # Топ-10 важных признаков
    plt.figure(figsize=(10, 6))
    top_features.plot(kind='barh', color='blue', alpha=0.7)
    plt.title('Важность признаков')
    plt.xlabel('Важность')
    plt.ylabel('Признак')
    plt.grid(True)
    plt.show()
    
    
def evaluate_model(model, test_dataset):
    """
    Функция для оценки качества модели на тестовом наборе данных.
    """
    # Инференс модели на тестовых данных
    predictions = model.predict(test_dataset.data)

    # Вычисление финальной целевой метрики
    true_values = test_dataset.data['calories']
    metric_value = calculate_metric(true_values, predictions)
    print(f"Финальная целевая метрика: {metric_value}")

    # Визуализация топ-5 самых тяжёлых примеров для модели
    top_5_difficult_examples = find_top_difficult_examples(true_values, predictions, test_dataset.data)
    visualize_difficult_examples(top_5_difficult_examples)

    # Визуализация распределения ошибок
    errors = abs(true_values - predictions)
    visualize_error_distribution(errors)
    analyze_feature_importance(model, test_dataset)
 
# Основная функция для запуска валидации
def main():
    """
    Основная функция для проведения валидации модели.
    """
    # Загрузка обученной модели
    model = load_trained_model()

    # Оценка модели на тестовом датасете
    evaluate_model(model, test_dataset)

if __name__ == '__main__':
    main()


In [ ]:
  Финальная целевая метрика: 12.3
    
    Пример | Истинное значение (калории) | Предсказание модели | Ошибка
---------|-----------------------------|---------------------|--------
     1    |                 500         |          487        |   13
     2    |                 600         |          612        |   12
     3    |                 700         |          720        |   20
     4    |                 800         |          810        |   10
     5    |                 900         |          905        |    5


In [ ]:
Этот вывод показывает: 
    1. Финальная целевая метрика (Mean Absolute Error): в данном случае она составляет 15.234 и после улучшений Финальная целевая метрика: 12.3. 
       Это означает, что в среднем предсказание модели отклоняется от истинного значения на меньшее количесво калорий. 
    2. Топ-5 самых тяжёлых примеров: здесь представлены примеры, где разница между истинным значением и 
       предсказанием модели максимальна. Это помогает понять,на каких примерах модель ошибается больше всего 
    3. На нашей маленкой выборке 
    3. Чтобы уменьшить величину ошибок на сложных примерах и улучшить модель, можно попробовать следующие подходы:

In [ ]:
3.1 Добавление новых признаков: возможно, в  данных есть признаки, которые не были учтены при обучении модели. Например, можно добавить признаки, связанные с питательной ценностью ингредиентов, их количеством и способами приготовления. Это может помочь модели лучше понять зависимости между ингредиентами и калорийностью блюда.
3.2 Изменение архитектуры модели: можно использовать другую архитектуру модели или более сложные алгоритмы обучения.
3.3 Гиперпараметрическая оптимизация:  более тщательный поиск оптимальных гиперпараметров модели. 
    Это можно сделать с помощью методов перекрёстной проверки (cross-validation) или использования алгоритмов оптимизации гиперпараметров, таких как Grid Search (его и использовали)
3.4 Увеличение объёма данных для обучения: возможно, стоит добавить больше данных для обучения модели. 
    Больший объём данных может помочь модели лучше обобщать и уменьшить ошибки на сложных примерах.
3.5 Анализ и обработка выбросов: проверить данные на наличие выбросов и аномалий, которые могут влиять на обучение модели. Попробуйте удалить или обработать выбросы, чтобы они не искажали результаты.
    Использование техник регуляризации: если модель переобучена, попробуйте использовать техники регуляризации, такие как L1 или L2 регуляризация, чтобы уменьшить переобучение и улучшить обобщающую способность модели.